<a href="https://colab.research.google.com/github/owaisahmad315/pandas/blob/main/Grouping%2C_Pivoting%2C_and_Reshaping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
# For this section we will use data representing student scores:
scores = pd.DataFrame({
    'name' : ['Adam', 'Bob', 'Dave', 'Fred'],
    'age' : [15, 16, 16, 15],
    'test1': [95, 81, 89, None],
    'test2': [80, 82, 84, 88],
    'teacher': ['Ashby', 'Ashby', 'Jones', 'Jones']
})

In [3]:
scores

,name,age,test1,test2,teacher
0,Adam,15,95.0,80,Ashby
1,Bob,16,81.0,82,Ashby
2,Dave,16,89.0,84,Jones
3,Fred,15,NaN,88,Jones


In [4]:
# Reducing Methods in groupby
scores.groupby('teacher').median()

<ipython-input-4-92673de8703e>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  scores.groupby('teacher').median()


,age,test1,test2
teacher,,,
Ashby,15.5,88.0,81.0
Jones,15.5,89.0,86.0


In [5]:
'''
This included the age column, to ignore that we can slice out just the
test columns:
'''
scores.groupby('teacher').median()[['test1', 'test2']]

<ipython-input-5-29bf71b58a2d>:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  scores.groupby('teacher').median()[['test1', 'test2']]


,test1,test2
teacher,,
Ashby,88.0,81.0
Jones,89.0,86.0


In [6]:
scores.groupby(['teacher', 'age']).median()
#When you group by multiple columns, the result has a hierarchical
# index or multi-level index.

<ipython-input-6-d5f414b39e16>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  scores.groupby(['teacher', 'age']).median()


test1  test2
teacher age              
Ashby   15    95.0   80.0
        16    81.0   82.0
Jones   15     NaN   88.0
        16    89.0   84.0

In [7]:
'''
If we want both the minimum and maximum test scores by teacher, we
use the .agg method and pass in a list of functions to call:
'''
scores.groupby(['teacher', 'age']).agg([min,max])

name       test1       test2    
              min   max   min   max   min max
teacher age                                  
Ashby   15   Adam  Adam  95.0  95.0    80  80
        16    Bob   Bob  81.0  81.0    82  82
Jones   15   Fred  Fred   NaN   NaN    88  88
        16   Dave  Dave  89.0  89.0    84  84

In [8]:
# Pivot Tables
'''
Using a pivot table, we can generalize certain groupby behaviors. To get
the median teacher scores we can run the following:
'''
scores.pivot_table(index='teacher',
                   values=['test1', 'test2'],
                   aggfunc='median')

,test1,test2
teacher,,
Ashby,88.0,81
Jones,89.0,86


In [9]:
'''
If we want to aggregate by teacher and age, we simply use a list with
both of them for the index parameter:
'''
scores.pivot_table(index=['teacher', 'age'],
                   values=['test1', 'test2'],
                   aggfunc='median')

test1  test2
teacher age              
Ashby   15    95.0     80
        16    81.0     82
Jones   15     NaN     88
        16    89.0     84

In [10]:
'''
If we want to apply multiple functions, just use a list of them. Here, we
look at the minimum and maximum test scores by teacher:
'''
scores.pivot_table(index='teacher',
                   values=['test1', 'test2'],
                   aggfunc=[min, max])

min         max      
        test1 test2 test1 test2
teacher                        
Ashby    81.0    80  95.0    82
Jones    89.0    84  89.0    88

In [11]:
# Melting Data
pd.melt(scores, id_vars=['name', 'age'],
        value_vars=['test1', 'test2'])

,name,age,variable,value
0,Adam,15,test1,95.0
1,Bob,16,test1,81.0
2,Dave,16,test1,89.0
3,Fred,15,test1,NaN
4,Adam,15,test2,80.0
5,Bob,16,test2,82.0
6,Dave,16,test2,84.0
7,Fred,15,test2,88.0


In [15]:
# Converting Back to Wide
'''
Using a pivot table, we can go from long format to wide format. It is a
little more involved going in the reverse direction:
'''
long_df = pd.melt(scores, id_vars=['name', 'age'],
                  value_vars=['test1', 'test2'],
                  var_name='test', value_name='score')
'''
First, we pivot, using the dimensions as the index parameter, the name
of the fact column name as the columns parameter, and the fact column as
the values parameter:
'''

wide_df = long_df.pivot_table(index=['name', 'age'],
                              columns=['test'],
                              values=['score'])
wide_df

score      
test     test1 test2
name age            
Adam 15   95.0  80.0
Bob  16   81.0  82.0
Dave 16   89.0  84.0
Fred 15    NaN  88.0